In [22]:
import gurobipy as gp
from gurobipy import GRB
import itertools

# QP DNR (Convex models of distribution system reconfiguration, 2012)
# decision variables
    # p(i,j) : line i,j 에 흐르는 real power
    # q(i,j) : line i,j 에 흐르는 imaginary power
    # y(i,j)
    # z(i,j) : switch status
    
    # Create a new model
model = gp.Model("powerflow_CP") 

    # Define the sets of generators and buses (nodes)
Buses = [1,2,3]
NonsubstationBuses = [2,3]
SubstationBuses = [1]
Lines = [(1,2),(1,3),(2,1),(3,1), (2,3),(3,2)] 
Switches = [(1,2),(1,3),(2,1),(3,1), (2,3),(3,2)] 
NoSwitches = []

AdjacencyList = {1: [2,3], 2: [1,3], 3:[1,2]}

    # Parameters
P = {1 : 2.2, 2 : 1, 3 : 1} # P[1]은 사용되지 않음
Q = {1: 3, 2 : 2, 3 : 4} # Q[1]은 사용되지 않음
G = {(1,2) : 0, (2,1) : 0, (2,3) : 0, (3,2) : 0} # notused
B = {(1,2) : 10, (2,1) : 10, (2,3) : 10, (3,2) : 10} # notused
R = {(1,2) : 0.001, (2,1) : 0.001, (2,3) : 0.001, (3,2) : 0.001, (1,3) : 0.001, (3,1) :0.001}
TransferBuses = []
for bus in Buses:
    if P[bus] == 0 and Q[bus] == 0:
        TransferBuses.append(bus)
DistributedGeneratorBuses = []
for bus in Buses:
    if P[bus] < 0 or Q[bus] < 0:
        DistributedGeneratorBuses.append(bus)

In [37]:
import gurobipy as gp
from gurobipy import GRB
import pandapower as pp
import pandapower.networks as pn

import itertools
from Networks import ieee123

net = ieee123.ieee123() # P, Q를 일정값으로 하니까 풀림/ P, Q를 0으로 하면 loop 생김
#net = pn.case33bw()


#net = pn.create_cigre_network_mv(with_der=False)


IEEE network is loaded


c:\Users\tmdck\Dropbox\5. Codes\7. optimizations\2. Reconfiguration\Networks\ieee123.py:102: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.sgen.type[net.sgen.type == 'WT'] = 'WP'


In [2]:
net.line

for i in net.line.index:
    if True:
        pp.create_switch(net, net.line.loc[i].from_bus, i, et = "l", closed = True) # 모든 line에 switch 연결



In [38]:
Buses = list(net.bus.index)
SubstationBuses = list(net.ext_grid.bus)
NonsubstationBuses = list(net.bus.index)

for bus in SubstationBuses:
    NonsubstationBuses.remove(bus)

Lines = []
R = dict()
for idx in net.line.index:
    col = net.line.loc[idx]
    Lines.append((col.from_bus, col.to_bus))
    Lines.append((col.to_bus, col.from_bus))
    R[(col.from_bus, col.to_bus)] = col.r_ohm_per_km*col.length_km
    R[(col.to_bus, col.from_bus)] = col.r_ohm_per_km*col.length_km
# switch 만 연결된 line 도 line임

for idx in net.switch.index:
    col = net.switch.loc[idx]
    if col.et == 'b':
        Lines.append((col.bus, col.element))
        Lines.append((col.element, col.bus))
        R[(col.bus, col.element)] = 0.00001
        R[(col.element, col.bus)] = 0.00001

    if col.et == 't':
        from_bus = net.trafo.loc[net.switch.loc[idx].element].hv_bus
        to_bus = net.trafo.loc[net.switch.loc[idx].element].lv_bus
        
        if (from_bus, to_bus) not in Lines:
            Lines.append((from_bus, to_bus))
        if (to_bus, from_bus) not in Lines:
            Lines.append((to_bus, from_bus))
        
        R[(to_bus, from_bus)] = 0.00001
        R[(from_bus, to_bus)] = 0.00001
        
sw = dict()
for idx in net.switch.index:
    
    from_bus = net.switch.loc[idx].bus
    to_bus = None
    if net.switch.loc[idx].et == 'l':
        if net.line.loc[net.switch.loc[idx].element].to_bus != from_bus:
            to_bus = net.line.loc[net.switch.loc[idx].element].to_bus
        else:
            to_bus = net.line.loc[net.switch.loc[idx].element].from_bus
    if net.switch.loc[idx].et == 'b':
        to_bus = net.switch.loc[idx].element
    
    if net.switch.loc[idx].et == 't':
        to_bus = net.trafo.loc[net.switch.loc[idx].element].lv_bus
        
    sw[idx] = (from_bus, to_bus)
    
Switches = []
for switch in sw.values():
    if switch not in Switches:
        Switches.append(switch)
    if tuple(reversed(switch)) not in Switches:
        Switches.append(tuple(reversed(switch)))

from copy import deepcopy

Switches = []
for switch in sw.values():
    if switch not in Switches:
        Switches.append(switch)
    if tuple(reversed(switch)) not in Switches:
        Switches.append(tuple(reversed(switch)))

NoSwitches = deepcopy(Lines)

for switch in Switches:
    NoSwitches.remove(switch)


def build_adjacency_list(edges):
    adjacency_list = {}
    for node1, node2 in edges:
        if node1 not in adjacency_list:
            adjacency_list[node1] = []
        if node2 not in adjacency_list:
            adjacency_list[node2] = [] 
        #adjacency_list[node1].append(node2)
        adjacency_list[node2].append(node1)  # If the graph is undirected

    return adjacency_list
AdjacencyList = build_adjacency_list(Lines)


P = dict()
Q = dict()
for bus in Buses:
    P[bus] = 0
    Q[bus] = 0
    
for idx in net.load.index:
    bus = net.load.bus.loc[idx]
    p = net.load.p_mw.loc[idx]*net.load.scaling.loc[idx]
    q = net.load.q_mvar.loc[idx]*net.load.scaling.loc[idx]
    
    P[bus] += p
    Q[bus] += q
    
for idx in net.sgen.index:
    bus = net.sgen.bus.loc[idx]
    p = net.sgen.p_mw.loc[idx]*net.sgen.scaling.loc[idx]
    q = net.sgen.q_mvar.loc[idx]*net.sgen.scaling.loc[idx]
    
    P[bus] -= p
    Q[bus] -= q
    
TransferBuses = []
for bus in Buses:
    if P[bus] == 0 and Q[bus] == 0:
        TransferBuses.append(bus)

DistributedGeneratorBuses = []
for bus in Buses:
    if P[bus] < 0 or Q[bus] < 0:
        DistributedGeneratorBuses.append(bus)

In [14]:
model = gp.Model("MIQP_DNR") 

p,q,y,z = dict(), dict(), dict(), dict()
for item in Lines:
    p[item] = model.addVar(name = 'p'+str(item))
    q[item] = model.addVar(name = 'q'+str(item))
    z[item] = model.addVar(name = 'z'+str(item))

for line in Switches:
    y[line] = model.addVar(vtype=GRB.BINARY,name = 'y'+str(line)) # eq 16

    
# Equality Constraint
for bus in Buses: # eq5
    if not bus in SubstationBuses:
        model.addConstr(
            sum(
                p[(adjbus, bus)] - p[(bus, adjbus)] for adjbus in AdjacencyList[bus]
                ) == P[bus])

for bus in Buses: #eq6
    if not bus in SubstationBuses:
        model.addConstr(
            sum(
                q[(adjbus, bus)] - q[(bus, adjbus)] for adjbus in AdjacencyList[bus]
                ) == Q[bus])

''' # eq 7, 8이나 없어야 돔
for bus in Buses: # eq7
    if bus in SubstationBuses:
        model.addConstr(
            sum(
                p[bus,adjbus] for adjbus in AdjacencyList[bus]
            ) == P[bus]
        )

for bus in Buses: #eq 8
    if bus in SubstationBuses:
        model.addConstr(
            sum(
                p[bus,adjbus] for adjbus in AdjacencyList[bus]
            ) == Q[bus]
        )
        '''
# Nonequality Constraint
M = 10000000000
for line in Lines: 
    model.addConstr(p[line] >= 0) # eq 9 
    model.addConstr(p[line] <= M*z[line]) # eq 9
    model.addConstr(q[line] >= 0) # eq 10
    model.addConstr(q[line] <= M*z[line]) # eq 10

    model.addConstr(z[line] >= 0) # eq 11
    if line[1] in SubstationBuses:
        model.addConstr(z[line] == 0) # eq 12

for line in NoSwitches:
    model.addConstr(z[line] + z[tuple(reversed(line))] == 1) # eq 13

for line in Switches:
    model.addConstr(z[line] + z[tuple(reversed(line))] == y[line]) # eq 14
    
for bus in Buses:
    if not bus in SubstationBuses:
        model.addConstr(
            sum(
                z[(adjbus, bus)] for adjbus in AdjacencyList[bus]
            ) == 1 
        ) # eq 15

model.setObjective(gp.quicksum(R[line]*(p[line]**2+q[line]**2) for line in Lines), GRB.MINIMIZE) # eq 4

model.update()

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 44 rows, 24 columns and 82 nonzeros
Model fingerprint: 0x1633182e
Model has 12 quadratic objective terms
Variable types: 18 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-03, 2e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 35 rows and 18 columns
Presolve time: 0.00s
Presolved: 9 rows, 6 columns, 20 nonzeros
Presolved model has 4 quadratic objective terms
Variable types: 0 continuous, 6 integer (2 binary)
Found heuristic solution: objective 0.0220000

Root relaxation: cutoff, 

In [39]:
model = gp.Model("MIQP_DNR") 

DistributedGeneratorBuses = []
for bus in Buses:
    if P[bus] < 0 or Q[bus] < 0:
        DistributedGeneratorBuses.append(bus)

p,q,x,y,k = dict(), dict(), dict(), dict(), dict()
for item in Lines:
    p[item] = model.addVar(name = 'p'+str(item))
    q[item] = model.addVar(name = 'q'+str(item))
    x[item] = model.addVar(vtype=GRB.BINARY, name = 'x'+str(item))
    k[item] = model.addVar(name = 'k'+str(item))

for bus in TransferBuses:
    y[bus] = model.addVar(vtype=GRB.BINARY, name = 'y'+str(bus)) # eq 16

K = dict()
for bus in Buses:
    if bus in DistributedGeneratorBuses:
        K[bus] = 1
    else:
        K[bus] = 0


for bus in Buses: # eq5
    if not bus in SubstationBuses:
        model.addConstr(
            sum(
                p[(adjbus, bus)] - p[(bus, adjbus)] for adjbus in AdjacencyList[bus]
                ) == P[bus])

for bus in Buses: #eq6
    if not bus in SubstationBuses:
        model.addConstr(
            sum(
                q[(adjbus, bus)] - q[(bus, adjbus)] for adjbus in AdjacencyList[bus]
                ) == Q[bus])


#for bus in NonsubstationBuses:
#    model.addConstr(
#        -P[bus] - sum(x[(bus, adjbus)]*p[(bus, adjbus)] for adjbus in AdjacencyList[bus]) == 0
#    )
    
#for bus in NonsubstationBuses:
#    model.addConstr(
#        -Q[bus] - sum(x[(bus, adjbus)]*p[(bus, adjbus)] for adjbus in AdjacencyList[bus]) == 0
#    )
    
    
M = 10000000000
for line in Lines: 
    model.addConstr(p[line] >= 0) # eq 9 
    model.addConstr(p[line] <= M*x[line]) # eq 9
    model.addConstr(q[line] >= 0) # eq 10
    model.addConstr(q[line] <= M*x[line]) # eq 10

    model.addConstr(x[line] >= 0) # eq 11


model.addConstr(
    sum(x[line] for line in Lines) == len(Buses) - len(SubstationBuses) - sum(1 - y[bus] for bus in TransferBuses)
)
for bus in TransferBuses:
    for adjbus in AdjacencyList[bus]:
        model.addConstr(
            x[(adjbus, bus)] <= y[bus]
        )
        model.addConstr(
            x[(bus, adjbus)] <= y[bus]
        )

    model.addConstr(
        sum(x[(bus, adjbus)] for adjbus in AdjacencyList[bus]) + sum(x[(adjbus, bus)] for adjbus in AdjacencyList[bus]) 
        >= 2*y[bus]
    )

for bus in Buses:
    model.addConstr(
        sum(k[adjbus, bus] for adjbus in AdjacencyList[bus])
        - sum(k[bus, adjbus] for adjbus in AdjacencyList[bus])
        == K[bus]
    )

for line in Lines:
    model.addConstr(
        k[line] <= len(DistributedGeneratorBuses)*x[line]
    )
    model.addConstr(
        -k[line] >= len(DistributedGeneratorBuses)*x[line]
    )

model.setObjective(gp.quicksum(R[line]*(p[line]**2+q[line]**2) for line in Lines), GRB.MINIMIZE) # eq 4

model.update()

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [36]:
model.getVars()

[<gurobi.Var p(1, 2) (value 1.0)>,
 <gurobi.Var q(1, 2) (value 2.666666666666667)>,
 <gurobi.Var x(1, 2) (value 1.0)>,
 <gurobi.Var k(1, 2) (value 0.0)>,
 <gurobi.Var p(1, 3) (value 1.0)>,
 <gurobi.Var q(1, 3) (value 3.3333333333333335)>,
 <gurobi.Var x(1, 3) (value 3.3333333333333337e-10)>,
 <gurobi.Var k(1, 3) (value 0.0)>,
 <gurobi.Var p(2, 1) (value 0.0)>,
 <gurobi.Var q(2, 1) (value 0.0)>,
 <gurobi.Var x(2, 1) (value -0.0)>,
 <gurobi.Var k(2, 1) (value 0.0)>,
 <gurobi.Var p(3, 1) (value 0.0)>,
 <gurobi.Var q(3, 1) (value 0.0)>,
 <gurobi.Var x(3, 1) (value -0.0)>,
 <gurobi.Var k(3, 1) (value 0.0)>,
 <gurobi.Var p(2, 3) (value 0.0)>,
 <gurobi.Var q(2, 3) (value 0.6666666666666665)>,
 <gurobi.Var x(2, 3) (value 6.666666666666666e-11)>,
 <gurobi.Var k(2, 3) (value 0.0)>,
 <gurobi.Var p(3, 2) (value 0.0)>,
 <gurobi.Var q(3, 2) (value 0.0)>,
 <gurobi.Var x(3, 2) (value 0.9999999996)>,
 <gurobi.Var k(3, 2) (value 0.0)>]

In [4]:
P

{0: 1,
 1: 1,
 2: 0,
 3: 0.7,
 4: 1,
 5: 1,
 6: 1,
 7: 0,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 0,
 13: 0,
 14: 0,
 15: 1,
 16: 0.7,
 17: 0,
 18: 1,
 19: 0.7,
 20: 0,
 21: 1,
 22: 0,
 23: 1,
 24: 0,
 25: 0,
 26: -0.3,
 27: 1,
 28: 1,
 29: 0.7,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 0,
 36: 1,
 37: 1,
 38: 0.7,
 39: 0,
 40: 1,
 41: 0.7,
 42: 1,
 43: 0,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 0.7,
 50: 1,
 51: 1,
 52: 1,
 53: 0,
 54: 1,
 55: 1,
 56: -0.3,
 57: 1,
 58: 1,
 59: 1,
 60: 0,
 61: 1,
 62: 1,
 63: 0.7,
 64: 1,
 65: 1,
 66: 0,
 67: 1,
 68: 1,
 69: 0.7,
 70: 1,
 71: 0,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 0,
 78: 0.7,
 79: 1,
 80: 0,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 0.7,
 87: 1,
 88: 0,
 89: 1,
 90: 0,
 91: 1,
 92: 0,
 93: 1,
 94: 0.7,
 95: 1,
 96: -0.3,
 97: 1,
 98: 1,
 99: 1,
 100: 0,
 101: 1,
 102: 1,
 103: 1,
 104: 0,
 105: 1,
 106: 1,
 107: 0,
 108: 1,
 109: 0,
 110: 1,
 111: 1,
 112: 1,
 113: 0.7,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119

In [5]:
import numpy as np
aaa = np.zeros(shape = (len(net.switch)))
for idx, switch in enumerate(Switches):
    aaa[idx//2] = y[switch].X
net.switch.closed = np.round(aaa).astype(int)

In [6]:
net.switch

,bus,element,et,type,closed,name,z_ohm,in_ka
0,0,0,l,None,1,None,0.0,NaN
1,0,1,l,None,1,None,0.0,NaN
2,0,2,l,None,1,None,0.0,NaN
3,2,3,l,None,1,None,0.0,NaN
4,2,4,l,None,1,None,0.0,NaN
5,4,5,l,None,1,None,0.0,NaN
6,6,6,l,None,1,None,0.0,NaN
7,7,7,l,None,1,None,0.0,NaN
8,7,8,l,None,1,None,0.0,NaN
9,7,9,l,None,0,None,0.0,NaN
